In [6]:
import pdb
import pycm
import sys
sys.path.append('../')
from glob import glob
import torch
import scipy
from torch import nn
import cv2
from tqdm import tqdm_notebook as tqdm
import os
from models import Model
from PIL import Image
from skimage import io, transform
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
import torchvision
from torchvision.datasets.folder import pil_loader
import random
import numpy as np
from torchsummary import summary
import pretrainedmodels
from sklearn.model_selection import KFold, StratifiedKFold
from efficientnet_pytorch import EfficientNet
import albumentations
from pycm import ConfusionMatrix
from tensorboard_logger import * 
import warnings
from dataloader import provider
warnings.filterwarnings("ignore")
from models import *
from image_utils import *
import albumentations
from albumentations import torch as AT
os.environ['CUDA_VISIBLE_DEVICES']="" # useful when testing dataloader
%matplotlib inline


In [3]:
name = "efficientnet-b5"
model = EfficientNet.from_pretrained(name, num_classes=5)

#in_features = model._fc.in_features
#model._fc = nn.Linear(in_features=in_features,
                      #out_features=out_features, bias=True)




Loaded pretrained weights for efficientnet-b5


In [4]:
model

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 48, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_bn0): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        48, 48, kernel_size=(3, 3), stride=[1, 1], groups=48, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        48, 12, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        12, 48, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        48, 24, kernel_siz

In [5]:
summary(model, input_size=(3, 300, 300), device="cpu")

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         ZeroPad2d-1          [-1, 3, 301, 301]               0
Conv2dStaticSamePadding-2         [-1, 48, 150, 150]           1,296
       BatchNorm2d-3         [-1, 48, 150, 150]              96
         ZeroPad2d-4         [-1, 48, 152, 152]               0
Conv2dStaticSamePadding-5         [-1, 48, 150, 150]             432
       BatchNorm2d-6         [-1, 48, 150, 150]              96
          Identity-7             [-1, 48, 1, 1]               0
Conv2dStaticSamePadding-8             [-1, 12, 1, 1]             588
          Identity-9             [-1, 12, 1, 1]               0
Conv2dStaticSamePadding-10             [-1, 48, 1, 1]             624
         Identity-11         [-1, 48, 150, 150]               0
Conv2dStaticSamePadding-12         [-1, 24, 150, 150]           1,152
      BatchNorm2d-13         [-1, 24, 150, 150]              48
      MBConv

In [8]:
model = pretrainedmodels.inceptionresnetv2(num_classes=1000, pretrained=None)

In [22]:
import torch
import torch.nn as nn


class BasicConv2d(nn.Module):

    def __init__(self, in_planes, out_planes, kernel_size, stride, padding=0):
        super(BasicConv2d, self).__init__()
        self.conv = nn.Conv2d(in_planes, out_planes, kernel_size=kernel_size, stride=stride,
                              padding=padding, bias=False)
        self.bn = nn.BatchNorm2d(out_planes, eps=0.001, momentum=0, affine=True)
        self.relu = nn.ReLU(inplace=False)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)
        return x


class StemBlock(nn.Module):
    '''
    input 299*299*3
    output 35*35*384
    '''
    def __init__(self):
        super(StemBlock, self).__init__()
        self.model_a = nn.Sequential(
            BasicConv2d(3, 32, kernel_size=3, stride=2),
            BasicConv2d(32, 32, kernel_size=3, stride=1, padding=1),
            BasicConv2d(32, 64, kernel_size=3, stride=1)
        )
        self.branch_a0 = nn.MaxPool2d(3, stride=2)
        self.branch_a1 = BasicConv2d(64, 96, kernel_size=3, stride=2)
        self.branch_b0 = nn.Sequential(
            BasicConv2d(160, 64, kernel_size=1, stride=1),
            BasicConv2d(64, 96, kernel_size=3, stride=1)
        )
        self.branch_b1 = nn.Sequential(
            BasicConv2d(160, 64, kernel_size=1, stride=1),
            BasicConv2d(64, 64, kernel_size=(1, 7), stride=1, padding=(0, 3)),
            BasicConv2d(64, 64, kernel_size=(7, 1), stride=1, padding=(3, 0)),
            BasicConv2d(64, 96, kernel_size=3, stride=1)
        )
        self.branch_c0 = BasicConv2d(192, 192, kernel_size=3, stride=2)
        self.branch_c1 = nn.MaxPool2d(3, stride=2)

    def forward(self, x):
        x = self.model_a(x)
        x_0 = self.branch_a0(x)
        x_1 = self.branch_a1(x)
        x = torch.cat((x_0, x_1), 1)
        x_0 = self.branch_b0(x)
        x_1 = self.branch_b1(x)
        x = torch.cat((x_0, x_1), 1)
        x_0 = self.branch_c0(x)
        x_1 = self.branch_c1(x)
        x = torch.cat((x_0, x_1), 1)
        return x


class InceptionResA(nn.Module):
    '''
    input 35*35*384
    output 35*35*384
    '''

    def __init__(self, scale=1.0):
        super(InceptionResA, self).__init__()
        self.relu = nn.ReLU(inplace=False)
        self.scale = scale
        self.branch_0 = BasicConv2d(384, 32, kernel_size=1, stride=1)
        self.branch_1 = nn.Sequential(
            BasicConv2d(384, 32, kernel_size=1, stride=1),
            BasicConv2d(32, 32, kernel_size=3, stride=1, padding=1)
        )
        self.branch_2 = nn.Sequential(
            BasicConv2d(384, 32, kernel_size=1, stride=1),
            BasicConv2d(32, 48, kernel_size=3, stride=1, padding=1),
            BasicConv2d(48, 64, kernel_size=3, stride=1, padding=1)
        )
        self.branch_all = BasicConv2d(128, 384, kernel_size=1, stride=1)

    def forward(self, x):
        x = self.relu(x)
        x_0 = self.branch_0(x)
        x_1 = self.branch_1(x)
        x_2 = self.branch_2(x)
        x_new = torch.cat((x_0, x_1, x_2), 1)
        x_new = self.branch_all(x_new)
        x = x + x_new * self.scale
        return x


class ReductionA(nn.Module):
    '''
    input 35*35*384
    output 17*17*1152
    '''
    def __init__(self):
        super(ReductionA, self).__init__()
        self.branch_0 = nn.MaxPool2d(3, stride=2)
        self.branch_1 = BasicConv2d(384, 384, kernel_size=3, stride=2)
        self.branch_2 = nn.Sequential(
            BasicConv2d(384, 256, kernel_size=1, stride=1),
            BasicConv2d(256, 256, kernel_size=3, stride=1, padding=1),
            BasicConv2d(256, 384, kernel_size=3, stride=2)
        )

    def forward(self, x):
        x_0 = self.branch_0(x)
        x_1 = self.branch_1(x)
        x_2 = self.branch_2(x)
        return torch.cat((x_0, x_1, x_2), 1)


class InceptionResB(nn.Module):
    '''
    input 17*17*1152
    output 17*17*1152
    '''
    def __init__(self, scale=1.0):
        super(InceptionResB, self).__init__()
        self.relu = nn.ReLU(inplace=False)
        self.scale = scale
        self.branch_0 = BasicConv2d(1152, 192, kernel_size=1, stride=1)
        self.branch_1 = nn.Sequential(
            BasicConv2d(1152, 128, kernel_size=1, stride=1),
            BasicConv2d(128, 160, kernel_size=(1, 7), stride=1, padding=(0, 3)),
            BasicConv2d(160, 192, kernel_size=(7, 1), stride=1, padding=(3, 0))
        )
        self.branch_all = BasicConv2d(384, 1152, kernel_size=1, stride=1)

    def forward(self, x):
        x = self.relu(x)
        x_0 = self.branch_0(x)
        x_1 = self.branch_1(x)
        x_new = torch.cat((x_0, x_1), 1)
        x_new = self.branch_all(x_new)
        x = x + x_new * self.scale
        return x


class ReductionB(nn.Module):
    '''
    input 17*17*1152
    ouput 8*8*2144
    '''
    def __init__(self):
        super(ReductionB, self).__init__()
        self.branch_0 = nn.MaxPool2d(3, stride=2)
        self.branch_1 = nn.Sequential(
            BasicConv2d(1152, 256, kernel_size=1, stride=1),
            BasicConv2d(256, 384, kernel_size=3, stride=2)
        )
        self.branch_2 = nn.Sequential(
            BasicConv2d(1152, 256, kernel_size=1, stride=1),
            BasicConv2d(256, 288, kernel_size=3, stride=2)
        )
        self.branch_3 = nn.Sequential(
            BasicConv2d(1152, 256, kernel_size=1, stride=1),
            BasicConv2d(256, 288, kernel_size=3, stride=1, padding=1),
            BasicConv2d(288, 320, kernel_size=3, stride=2)
        )

    def forward(self, x):
        x_0 = self.branch_0(x)
        x_1 = self.branch_1(x)
        x_2 = self.branch_2(x)
        x_3 = self.branch_3(x)
        return torch.cat((x_0, x_1, x_2, x_3), 1)


class InceptionResC(nn.Module):
    '''
    input 8*8*2144
    output 8*8*2144
    '''
    def __init__(self, scale=1.0):
        super(InceptionResC, self).__init__()
        self.scale = scale
        self.relu = nn.ReLU(inplace=False)
        self.branch_0 = BasicConv2d(2144, 192, kernel_size=1, stride=1)
        self.branch_1 = nn.Sequential(
            BasicConv2d(2144, 192, kernel_size=1, stride=1),
            BasicConv2d(192, 224, kernel_size=(1, 3), stride=1, padding=(0, 1)),
            BasicConv2d(224, 256, kernel_size=(3, 1), stride=1, padding=(1, 0))
        )
        self.branch_all = BasicConv2d(448, 2144, kernel_size=1, stride=1)

    def forward(self, x):
        x = self.relu(x)
        x_0 = self.branch_0(x)
        x_1 = self.branch_1(x)
        x_new = torch.cat((x_0, x_1), 1)
        x_new = self.branch_all(x_new)
        x = x + x_new * self.scale
        return x


class InceptionResV2(nn.Module):

    def __init__(self, num_classes=1000):
        super(InceptionResV2, self).__init__()
        self.stem = StemBlock()
        self.inception_resA5 = nn.Sequential(
            InceptionResA(),
            InceptionResA(),
            InceptionResA(),
            InceptionResA(),
            InceptionResA()
        )
        self.reductionA = ReductionA()
        self.inception_resB10 = nn.Sequential(
            InceptionResB(),
            InceptionResB(),
            InceptionResB(),
            InceptionResB(),
            InceptionResB(),
            InceptionResB(),
            InceptionResB(),
            InceptionResB(),
            InceptionResB(),
            InceptionResB()
        )
        self.reductionB = ReductionB()
        self.inception_resC5 = nn.Sequential(
            InceptionResC(),
            InceptionResC(),
            InceptionResC(),
            InceptionResC(),
            InceptionResC()
        )
        self.avg_pool = nn.AvgPool2d(8, count_include_pad=False)
        self.dropout = nn.Dropout2d(p=0.8)
        self.fc = nn.Linear(2144, num_classes)

    def forward(self, x):
        x = self.stem(x)
        x = self.inception_resA5(x)
        x = self.reductionA(x)
        x = self.inception_resB10(x)
        x = self.reductionB(x)
        x = self.inception_resC5(x)
        x = self.avg_pool(x)
        x = self.dropout(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [23]:
# model = pretrainedmodels.inceptionv4(num_classes=1000, pretrained=None)
model = InceptionResV2(num_classes=5)

In [24]:
summary(model, input_size=(3, 299, 299), device="cpu")

torch.Size([2, 2144, 8, 8])
torch.Size([2, 2144, 8, 8])
torch.Size([2, 2144, 8, 8])
torch.Size([2, 2144, 8, 8])
torch.Size([2, 2144, 8, 8])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 149, 149]             864
       BatchNorm2d-2         [-1, 32, 149, 149]              64
              ReLU-3         [-1, 32, 149, 149]               0
       BasicConv2d-4         [-1, 32, 149, 149]               0
            Conv2d-5         [-1, 32, 149, 149]           9,216
       BatchNorm2d-6         [-1, 32, 149, 149]              64
              ReLU-7         [-1, 32, 149, 149]               0
       BasicConv2d-8         [-1, 32, 149, 149]               0
            Conv2d-9         [-1, 64, 147, 147]          18,432
      BatchNorm2d-10         [-1, 64, 147, 147]             128
             ReLU-11         [-1, 64, 147, 147]               0
      BasicConv2d-12       

In [ ]:

nception resnet c 5 elt